<h1>Product page parser</h1>

<p>In this notebook we are going to parse a single product page.</p>
<p>According to the task we should get the following: </p>
<ul>
  <li>categories</li>
  <li>subcategories</li> 
  <li>name</li>
  <li>price</li>
  <li>description</li>
  <li>charachteristics</li>
  <li>similar products</li>
  <li>product image link</li>
  <li>if product is a hit or not</li>
</ul> 

<h3><i>Important note</i></h3>
<p>Categories and subcategories (both level 1 and level 2) we plan to extract from parsing table obtained from the catalog main page (the parsing table currently contains 1222 rows. Each row has a link to products page with multiple product cards).</p>

<p>So from the product page we are going to extract the following:</p>
<ol>
  <li>name</li>
  <li>price</li>
  <li>description</li>
  <li>charachteristics</li>
  <li>similar products</li>
  <li>product image link</li>
  <li>if product is a hit or not</li>
</ol> 


In [5]:
import requests
import urllib.request
import time
from bs4 import BeautifulSoup as BSoup
import pandas as pd
import re
import lxml
import html5lib

<p>We will develop our parser on a bosch drill webpage which seems to be the most complete and contains all the features, including hit tag.</p>

In [6]:
#url = 'https://www.oma.by/tros-dlya-rastyazhki-din3055-d1-5-mm-1-238091-p'
url = 'https://www.oma.by/perforator-bosch-pbh-2100-re-2-zubila-2-bura-1-216760-p'
single_product_page = requests.get(url)
product_page_soup = BSoup(single_product_page.text, 'html.parser')

product_page_special_soup = BSoup(single_product_page.text, 'html5lib')

<p style="color:green"><b>1. name </b></p>

In [7]:
product_name_raw = product_page_soup.select('div.page-title h1')
product_name = product_name_raw[0].text
print(product_name)

Перфоратор BOSCH PBH 2100 RE+2 зубила, 2 бура, Минск 


<p style="color:green"><b>2. price </b></p>

In [8]:
product_price_raw = product_page_soup.select('div.product-info-box_price')
product_price_text_raw = product_price_raw[0].text
product_price_text = product_price_text_raw.replace(" ", "")
product_price = str(product_price_text[:7])
print(product_price)


199,00


<p style="color:green"><b>3. description</b></p>

In [9]:
product_description_raw = product_page_soup.select('article.catalog-item-description-txt_content')
product_description = product_description_raw[0].text
print(product_description)


                                    BOSCH PBH 2100 RE универсальный и эргономичный перфоратор для бытового использования. Имеет 3 режима работы: сверление, долбление и бурение. Развивает скорость до 2300 об/мин. Патрон SDS-Plus позволяет быстро менять насадки. Оснащен функцией блокировки удара через кнопку выключения. Доступно завинчивание/вывинчивание шурупов через режим реверса. Дополнительная рукоятка обеспечивает равномерное распределение нагрузки. В комплекте предусмотрен ограничитель глубины для регулировки размера отверстий. Поставляется в чемодане с 2 зубилами и 2 бурами.                                


<p style="color:green"><b>4. characteristics </b></p>

In [10]:
product_characteristics_raw = product_page_soup.select('div.params-blocks')
if len(product_characteristics_raw) != 0:
    product_characteristics = product_characteristics_raw[0].text
    print(product_characteristics)
else:
    print('No parameters found')



Общие характеристики



Режим работы - сверление




Режим работы - сверление                                                                    
Возможность сверления.




Да



Режим работы - долбление

Да



Режимы работы - сверление с ударом

Да



Количество скоростей работы

1



Потребляемая мощность

550



Макс. число оборотов холостого хода

2300



Макс. количество ударов в минуту

5800



Макс. энергия удара

1.7



Питание

сеть




Технические характеристики



Шуруповерт

Да



Тип крепления бура

SDS-Plus



Макс. диаметр сверления (дерево)

30



Макс. диаметр сверления (металл)

13



Макс. диаметр сверления (бетон)

20




Функциональные особенности



Блокировка кнопки включения

Да



Фиксация шпинделя

Да



Электронная регулировка частоты вращения

Да




Комплектация



Реверс

Да



Дополнительная рукоятка

Да



Кейс в комплекте

Да



Ограничитель глубины сверления

Да



Комплектация

2 бура2 зубилаглубиномеринструкция по эксплуатациичемодан




Размеры и 

<p style="color:red"><b>5. similar products </b></p>
<p>Failed to extract similar products.</p>

In [11]:
similar_products_raw = product_page_special_soup.find(id = "sales_shared")
#similar_products_raw = product_page_soup.findAll('div',{'class':'sales_shared'})
#similar > div
#product_name = product_name_raw[0].text
print(similar_products_raw)


None


<p style="color:green"><b>6. product image link </b></p>

In [12]:
product_image_link_raw = product_page_soup.select('div.slider-w-preview img')
product_image_link = 'https://www.oma.by' + product_image_link_raw[0].get('src')
print(product_image_link)

https://www.oma.by/upload/Sh/imageCache/03a/cc1/1.216760-medium.jpg


<p style="color:green"><b>7. if product is a hit or not </b></p>

In [13]:
hit_offer_raw = product_page_soup.findAll('span',{'class':'icon special-icon special-icon__hit product-item_special'})
print(hit_offer_raw)
if len(hit_offer_raw) != 0:
    product_is_hit = True
    print('\nThe product is hit!')
else:
    product_is_hit = False
    print('\nThe product is not hit.')

[<span class="icon special-icon special-icon__hit product-item_special"></span>]

The product is hit!


<h3>Create product table</h3>

In [14]:
product_df = pd.DataFrame(columns = ['name',\
                                     'price','description', 'characteristics', \
                                     'product image link', 'is_hit']) 
product_df

,name,price,description,characteristics,product image link,is_hit


In [15]:
product_df.loc[0] = [product_name] + [product_price]  \
                  + [product_description] + [product_characteristics] \
                  + [product_image_link] + [product_is_hit]
product_df.head()                       

,name,price,description,characteristics,product image link,is_hit
0,"Перфоратор BOSCH PBH 2100 RE+2 зубила, 2 бура,...","\n199,00",\n BOSCH PB...,\n\nОбщие характеристики\n\n\n\nРежим работы -...,https://www.oma.by/upload/Sh/imageCache/03a/cc...,True


<h3>Product parameters function</h3>
<p>Construct function that returns a dictionary with product parameters. That is sketchy version of a function. Later will be converted into normal view.</p> 

In [16]:
def get_product_parameters(soup):
    
    #product name
    product_name_raw = product_page_soup.select('div.page-title h1')
    product_name = product_name_raw[0].text
    
    #product_price
    product_price_raw = product_page_soup.select('div.product-info-box_price')
    product_price_text_raw = product_price_raw[0].text
    product_price_text = product_price_text_raw.replace(" ", "")
    product_price = str(product_price_text[:7])
    
    #product_description
    product_description_raw = product_page_soup.select('article.catalog-item-description-txt_content')
    product_description = product_description_raw[0].text
    
    #product_characteristics
    product_characteristics_raw = product_page_soup.select('div.params-blocks')
    if len(product_characteristics_raw) != 0:
        product_characteristics = product_characteristics_raw[0].text
    else:
        product_characteristics = ''
        
    #similar_products
    similar_products = ''
        
    #product_image_link    
    product_image_link_raw = product_page_soup.select('div.slider-w-preview img')
    product_image_link = 'https://www.oma.by' + product_image_link_raw[0].get('src')
    
    #product_is_hit
    hit_offer_raw = product_page_soup.findAll('span',{'class':'icon special-icon special-icon__hit product-item_special'})
    if len(hit_offer_raw) != 0:
        product_is_hit = True

    else:
        product_is_hit = False

                                                      
    dict = {
        'product_name' : product_name,
        'product_price' : product_price,
        'product_description' : product_description,
        'product_characteristics' : product_characteristics,
        'similar_products' : similar_products,
        'product_image_link' : product_image_link,
        'product_is_hit' : product_is_hit,
    }
    
    
    return dict
    

In [17]:
product_parameters_dict = get_product_parameters(product_page_soup)
for param in product_parameters_dict:
    print(f'\n\n{param} {product_parameters_dict[param]}')



product_name Перфоратор BOSCH PBH 2100 RE+2 зубила, 2 бура, Минск 


product_price 
199,00


product_description 
                                    BOSCH PBH 2100 RE универсальный и эргономичный перфоратор для бытового использования. Имеет 3 режима работы: сверление, долбление и бурение. Развивает скорость до 2300 об/мин. Патрон SDS-Plus позволяет быстро менять насадки. Оснащен функцией блокировки удара через кнопку выключения. Доступно завинчивание/вывинчивание шурупов через режим реверса. Дополнительная рукоятка обеспечивает равномерное распределение нагрузки. В комплекте предусмотрен ограничитель глубины для регулировки размера отверстий. Поставляется в чемодане с 2 зубилами и 2 бурами.                                


product_characteristics 

Общие характеристики



Режим работы - сверление




Режим работы - сверление                                                                    
Возможность сверления.




Да



Режим работы - долбление

Да



Режимы работы - сверление 

In [1]:
import parser

In [2]:
print_hello()

NameError: name 'print_hello' is not defined